### Projeto comparação entre algoritmos e modelos de Computação evolutiva

Achando o diretório em que esse arquivo se encontra 

In [15]:
import sys
import os

# Obtém o caminho do diretório do script atual
script_dir = os.getcwd()

# Adiciona o diretório pai ao caminho do sistema
sys.path.append(script_dir)

In [16]:
import pygame # Nos repositórios de referência essa lib é sempre importada tendo em vista que o gymnasium (gym) utiliza a mesma, deve evitar problemas

import time # será utilizada para coletar o período temporal necessário para que algumas funções sejam executadas
import numpy as np # numpy por motivos óbvios  

import torch # Todas as redes neurais serão criados utilizando PyTorch
import torch.nn as nn
import torch.optim as optim

from redes_neurais.rede_MLP import MLP
from func_aux.auxiliares import *

import matplotlib.pyplot as plt # Demonstração de Gráficos e imagens 

In [17]:
# usar GPU compatível com CUDA costuma ser mais rápido
if torch.cuda.is_available():
    is_gpu = torch.device("cuda")
else:
    is_gpu = torch.device("cpu")

torch.set_default_device(is_gpu)

In [18]:
import gymnasium as gym

In [19]:
def joga_jogo(env, rede_neural):

    if isinstance(env, str):
        env = gym.make(env)

    imagem, _ = env.reset()
    passos = 0
    game_recompenca = 0

    while True: # inicializa o loop do jogo
        passos += 1 # conta quantas vezes a IA foi requisitada ate o complecionismo do mesmo

        acao = escolhe_acao(rede_neural, imagem)

        nova_imagem, recompensa, finalizado, truncado, _ = env.step(acao)
        done = finalizado or truncado
        game_recompenca += recompensa

        imagem = nova_imagem

        if done:
            return rede_neural, passos, game_recompenca

In [20]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []
    recompencas = []

    recompencas_ao_tempo = []
    Tempo_por_geracao = []

    contador_recompencas = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        inicio = time.time()

        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
    
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        recompencas_ao_tempo.append(recompencas)
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        fim = time.time()
        Tempo_por_geracao.append(fim - inicio)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
    
    obs = [recompencas_ao_tempo, Tempo_por_geracao]
    return selecao_final[0], obs

In [21]:
env = gym.make("CartPole-v1")

redes_analisadas_1 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = True"
    }

    redes_analisadas_1.append([
        rede_melhor, informacoes_Acrobot
    ])


Geração: 1/20, Recompensa: [28.0, 25.0, 18.0]
Geração: 2/20, Recompensa: [52.0, 48.0, 37.0]
Geração: 3/20, Recompensa: [101.0, 66.0, 61.0]
Geração: 4/20, Recompensa: [105.0, 101.0, 91.0]
Geração: 5/20, Recompensa: [149.0, 105.0, 103.0]
Geração: 6/20, Recompensa: [149.0, 109.0, 105.0]
Geração: 7/20, Recompensa: [149.0, 142.0, 109.0]
Geração: 8/20, Recompensa: [149.0, 142.0, 129.0]
Geração: 9/20, Recompensa: [149.0, 142.0, 137.0]
Geração: 10/20, Recompensa: [149.0, 142.0, 137.0]
Geração: 11/20, Recompensa: [149.0, 142.0, 137.0]
Geração: 12/20, Recompensa: [300.0, 149.0, 142.0]
Geração: 13/20, Recompensa: [300.0, 210.0, 149.0]
Geração: 14/20, Recompensa: [300.0, 258.0, 225.0]
Geração: 15/20, Recompensa: [453.0, 300.0, 258.0]
Geração: 16/20, Recompensa: [453.0, 368.0, 300.0]
Geração: 17/20, Recompensa: [500.0, 453.0, 404.0]
Geração: 18/20, Recompensa: [500.0, 500.0, 453.0]
Geração: 1/20, Recompensa: [104.0, 33.0, 12.0]
Geração: 2/20, Recompensa: [135.0, 104.0, 56.0]
Geração: 3/20, Recompen

In [22]:
env = gym.make("CartPole-v1")

redes_analisadas_2 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = False,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = False"
    }

    redes_analisadas_2.append([
        rede_melhor, informacoes_Acrobot
    ])


Geração: 1/20, Recompensa: [29.0, 24.0, 14.0]
Geração: 2/20, Recompensa: [176.0, 68.0, 40.0]
Geração: 3/20, Recompensa: [176.0, 110.0, 102.0]
Geração: 4/20, Recompensa: [176.0, 110.0, 102.0]
Geração: 5/20, Recompensa: [500.0, 379.0, 254.0]
Geração: 6/20, Recompensa: [500.0, 379.0, 261.0]
Geração: 7/20, Recompensa: [500.0, 500.0, 500.0]
Geração: 1/20, Recompensa: [29.0, 18.0, 12.0]
Geração: 2/20, Recompensa: [159.0, 53.0, 53.0]
Geração: 3/20, Recompensa: [202.0, 159.0, 152.0]
Geração: 4/20, Recompensa: [202.0, 173.0, 159.0]
Geração: 5/20, Recompensa: [208.0, 202.0, 196.0]
Geração: 6/20, Recompensa: [208.0, 205.0, 202.0]
Geração: 7/20, Recompensa: [500.0, 208.0, 205.0]
Geração: 8/20, Recompensa: [500.0, 500.0, 500.0]
Geração: 1/20, Recompensa: [27.0, 18.0, 14.0]
Geração: 2/20, Recompensa: [149.0, 118.0, 83.0]
Geração: 3/20, Recompensa: [149.0, 126.0, 118.0]
Geração: 4/20, Recompensa: [169.0, 149.0, 126.0]
Geração: 5/20, Recompensa: [215.0, 169.0, 149.0]
Geração: 6/20, Recompensa: [215.0,

In [23]:
env = gym.make("Acrobot-v1")

redes_analisadas_3 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = -65
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = True"
    }

    redes_analisadas_3.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [-211.0, -238.0, -500.0]
Geração: 2/20, Recompensa: [-101.0, -211.0, -215.0]
Geração: 3/20, Recompensa: [-76.0, -101.0, -123.0]
Geração: 4/20, Recompensa: [-73.0, -76.0, -91.0]
Geração: 5/20, Recompensa: [-73.0, -76.0, -87.0]
Geração: 6/20, Recompensa: [-73.0, -76.0, -84.0]
Geração: 7/20, Recompensa: [-70.0, -72.0, -73.0]
Geração: 8/20, Recompensa: [-65.0, -70.0, -72.0]
Geração: 9/20, Recompensa: [-65.0, -65.0, -70.0]
Geração: 1/20, Recompensa: [-177.0, -397.0, -500.0]
Geração: 2/20, Recompensa: [-146.0, -149.0, -177.0]
Geração: 3/20, Recompensa: [-124.0, -124.0, -138.0]
Geração: 4/20, Recompensa: [-94.0, -119.0, -124.0]
Geração: 5/20, Recompensa: [-76.0, -82.0, -94.0]
Geração: 6/20, Recompensa: [-76.0, -82.0, -84.0]
Geração: 7/20, Recompensa: [-64.0, -76.0, -82.0]
Geração: 8/20, Recompensa: [-64.0, -67.0, -74.0]
Geração: 9/20, Recompensa: [-64.0, -66.0, -67.0]
Geração: 10/20, Recompensa: [-64.0, -64.0, -65.0]
Geração: 1/20, Recompensa: [-500.0, -500.0, -500.

In [24]:
env = gym.make("Acrobot-v1")

redes_analisadas_4 = []

for c in range(10):

    tempo_inicio = time.time()
    rede_melhor, obs = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = False,
        learning_rate = 0.1,
        objetivo = -65
    )
    Tempo_fim = time.time()


    informacoes_Acrobot = {
        'Nome do ambiente': "Acrobot",
        'Tempo Total': Tempo_fim - tempo_inicio,
        'recompencas por geracao': obs[0],
        'tempo por geracao': obs[1],
        'Tipo do algoritimo': "Pais = False"
    }

    redes_analisadas_4.append([
        rede_melhor, informacoes_Acrobot
    ])

Geração: 1/20, Recompensa: [-133.0, -378.0, -387.0]
Geração: 2/20, Recompensa: [-91.0, -133.0, -151.0]
Geração: 3/20, Recompensa: [-64.0, -82.0, -91.0]
Geração: 4/20, Recompensa: [-64.0, -65.0, -82.0]
Geração: 5/20, Recompensa: [-64.0, -65.0, -81.0]
Geração: 6/20, Recompensa: [-64.0, -65.0, -74.0]
Geração: 7/20, Recompensa: [-64.0, -65.0, -74.0]
Geração: 8/20, Recompensa: [-64.0, -65.0, -74.0]
Geração: 9/20, Recompensa: [-64.0, -65.0, -70.0]
Geração: 10/20, Recompensa: [-64.0, -65.0, -70.0]
Geração: 11/20, Recompensa: [-64.0, -64.0, -65.0]
Geração: 12/20, Recompensa: [-63.0, -64.0, -64.0]
Geração: 13/20, Recompensa: [-63.0, -64.0, -64.0]
Geração: 14/20, Recompensa: [-63.0, -64.0, -64.0]
Geração: 15/20, Recompensa: [-63.0, -63.0, -64.0]
Geração: 16/20, Recompensa: [-63.0, -63.0, -64.0]
Geração: 1/20, Recompensa: [-500.0, -500.0, -500.0]
Geração: 2/20, Recompensa: [-500.0, -500.0, -500.0]
Geração: 3/20, Recompensa: [-238.0, -364.0, -500.0]
Geração: 4/20, Recompensa: [-113.0, -190.0, -238

In [27]:
# env, env_name, gamma, qnet, qnet_lr, target_qnet, target_update_freq, replay_size, batch_size, epsilon_f, epsilon_decay_period, NUM_STAPS
def treinamento_Foda(env, rede_neural_hidden, geracoes, n_por_geracao, n_selecionados = 2, pais = False, learning_rate = 0.001, objetivo = 500):
    
    if isinstance(env, str):
        env = gym.make(env)
    
    individuos = []
    
    input_dim = env.observation_space.shape[0]  # Número de entradas do ambiente MountainCar-v0
    output_dim = env.action_space.n  # Número de ações possíveis no ambiente MountainCar-v0

    for indi in range(n_por_geracao):
        individuos.append(MLP(input_dim, rede_neural_hidden, output_dim))

    lista_geracional = []
    recompencas = []

    recompencas_ao_tempo = []
    Tempo_por_geracao = []

    contador_recompencas = 0
    contador_max_passos = 0

    for geracao in range(1,geracoes+1):
        contador = 0
        inicio = time.time()

        for especime_network in individuos:
            contador += 1

            modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
            lista_geracional.append([modelo_final, recompensa_total])
            recompencas.append(recompensa_total)

            if recompensa_total == objetivo:
                contador_recompencas += 1
            # print(f'Geração: {geracao}/{geracoes}, Individuo: {contador}/{n_por_geracao}, Recompença: {recompensa_total}')
    
        pais_selecionados = seleciona_pais(lista_geracional, n_selecionados)

        verificador = sorted(recompencas, reverse=True)[:3]
        contador_max_passos += passos
        if verificador[0] >= objetivo:
            return contador_max_passos
            

        print(f'Geração: {geracao}/{geracoes}, Recompensa: {sorted(recompencas, reverse=True)[:3]}')
        recompencas_ao_tempo.append(recompencas)
        if pais:
            individuos = computacao_evolutiva_mutador_Cpais(n_por_geracao, pais_selecionados, learning_rate)
        else:
            individuos = computacao_evolutiva_mutador_Spais(n_por_geracao, pais_selecionados, learning_rate)
        
        fim = time.time()
        Tempo_por_geracao.append(fim - inicio)
        
        if contador_recompencas >= 2:
            break
    
    for especime_network in individuos:
        contador += 1

        modelo_final, passos, recompensa_total = joga_jogo(env, especime_network)
        lista_geracional.append([modelo_final, recompensa_total])
        recompencas.append(recompensa_total)
        selecao_final = seleciona_pais(lista_geracional, 1)
    
    obs = [recompencas_ao_tempo, Tempo_por_geracao]
    return selecao_final[0], obs

In [53]:
env = gym.make("CartPole-v1")

redes_analisadas_5 = []
contador_n_finalizados = 0

for c in range(100):

    tempo_inicio = time.time()
    QTDs_passos = treinamento_Foda(
        env = env,
        rede_neural_hidden = [36,36],
        geracoes = 20,
        n_por_geracao = 30,
        n_selecionados = 6,
        pais = True,
        learning_rate = 0.1,
        objetivo = 500
    )
    Tempo_fim = time.time()
    print(QTDs_passos)
    if isinstance(QTDs_passos, int):
        redes_analisadas_5.append(QTDs_passos)
    else:
        contador_n_finalizados += 1
    
    print(f"Fim da geração {c+1}")


Geração: 1/20, Recompensa: [29.0, 19.0, 18.0]
Geração: 2/20, Recompensa: [135.0, 50.0, 48.0]
Geração: 3/20, Recompensa: [135.0, 67.0, 58.0]
Geração: 4/20, Recompensa: [319.0, 135.0, 87.0]
Geração: 5/20, Recompensa: [434.0, 342.0, 319.0]
625
Fim da geração 1
Geração: 1/20, Recompensa: [124.0, 58.0, 17.0]
Geração: 2/20, Recompensa: [124.0, 112.0, 65.0]
Geração: 3/20, Recompensa: [124.0, 112.0, 97.0]
Geração: 4/20, Recompensa: [412.0, 232.0, 145.0]
Geração: 5/20, Recompensa: [412.0, 322.0, 242.0]
Geração: 6/20, Recompensa: [412.0, 389.0, 370.0]
611
Fim da geração 2
Geração: 1/20, Recompensa: [16.0, 11.0, 10.0]
Geração: 2/20, Recompensa: [66.0, 48.0, 27.0]
Geração: 3/20, Recompensa: [327.0, 195.0, 147.0]
Geração: 4/20, Recompensa: [327.0, 246.0, 232.0]
Geração: 5/20, Recompensa: [327.0, 290.0, 277.0]
Geração: 6/20, Recompensa: [355.0, 327.0, 298.0]
844
Fim da geração 3
Geração: 1/20, Recompensa: [22.0, 12.0, 12.0]
Geração: 2/20, Recompensa: [94.0, 60.0, 30.0]
35
Fim da geração 4
Geração: 1

In [54]:
def calcular_media(lista):
    """Calcula a média de uma lista."""
    if not lista:
        return 0  # Retorna 0 se a lista estiver vazia para evitar divisão por zero
    return sum(lista) / len(lista)

In [62]:
print("A média da lista de passos é:", round(calcular_media(redes_analisadas_5)))
print(f"O maior numero de passos {max(redes_analisadas_5)}")
print(f"Não finalizados {contador_n_finalizados} logo o tempo é: {2500 * contador_n_finalizados}")

A média da lista de passos é: 460
O maior numero de passos 2283
Não finalizados 5 logo o tempo é: 12500


In [ ]:
# Salvar cada lista
for i, lista in enumerate([redes_analisadas_1, redes_analisadas_2, redes_analisadas_3, redes_analisadas_4, redes_analisadas_5], start=1):
    if lista != redes_analisadas_5:
        lista = [x for _, x in lista]
    nome_arquivo = f'redes_analisadas_{i}.pkl'
    salvar_dados(lista, nome_arquivo)

In [67]:
nome_arquivo = f'redes_analisadas_5.pkl'
lista_carregada = recarregar_dados(nome_arquivo)
print(f'Lista redes_analisadas_{i} recarregada:', lista_carregada)

Dados recarregados de redes_analisadas_5.pkl.
Lista redes_analisadas_5 recarregada: [625, 611, 844, 35, 392, 461, 205, 530, 785, 273, 420, 352, 106, 815, 61, 1104, 1267, 528, 1627, 125, 53, 300, 164, 678, 82, 147, 568, 422, 36, 180, 83, 437, 297, 1136, 273, 398, 1030, 1365, 29, 268, 251, 53, 49, 894, 616, 272, 419, 596, 301, 1410, 65, 2283, 488, 205, 291, 187, 204, 150, 30, 10, 401, 828, 120, 111, 567, 164, 481, 488, 1154, 176, 20, 236, 337, 38, 105, 440, 1076, 300, 91, 813, 212, 1656, 342, 718, 247, 241, 864, 225, 870, 1221, 58, 86, 657, 316, 112]
